In [ ]:
!pip install --upgrade transformers

In [1]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2025-04-12 13:26:41--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9462138 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   9.02M  18.6MB/s    in 0.5s    

2025-04-12 13:26:42 (18.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [9462138/9462138]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [3]:
!./ngrok authtoken ADD_AUTH_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
!huggingface-cli login 

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `ish` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ish`


In [5]:
from flask import Flask, request, jsonify
import threading

In [6]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
chat_history = [
    {
        "role": "system",
        "content": """You are a fantasy speech exercise generator who makes speech exercises on targetted phonemes and answers in a JSON format only.
Below is the list of all phonemes in SAPI format along with an examples for what they sound like, LEARN WHAT THE PHONEMES SOUND LIKE FROM THE EXAMPLES.
[
  // Vowels
  {
    "phoneme": "iy",
    "examples": ["seat", "beet", "peach", "mead", "eel", "cheat", "gleam", "seeker", "treat", "spirit"]
  },
  {
    "phoneme": "ih",
    "examples": ["sit", "bit", "flick", "grit", "whisper", "tick", "mystic", "quick", "illusion", "ring"]
  },
  {
    "phoneme": "eh",
    "examples": ["bed", "head", "elf", "net", "red", "quest", "pest", "spell", "feather", "trek"]
  },
  {
    "phoneme": "ae",
    "examples": ["cat", "bat", "dragon", "magic", "apple", "flash", "wand", "blast", "tavern", "trap"]
  },
  {
    "phoneme": "aa",
    "examples": ["father", "wand", "calm", "charm", "lava", "bard", "dark", "palace", "gnarled", "thar"]
  },
  {
    "phoneme": "ah",
    "examples": ["cup", "luck", "puff", "mug", "thunder", "grumble", "summon", "blood", "mud", "bunk"]
  },
  {
    "phoneme": "uh",
    "examples": ["book", "hook", "look", "foot", "crook", "pudding", "goblin", "brook", "wood", "push"]
  },
  {
    "phoneme": "uw",
    "examples": ["boot", "moon", "rune", "loop", "doom", "fool", "broom", "soothe", "gloom", "blue"]
  },
  {
    "phoneme": "er",
    "examples": ["her", "fur", "whirl", "curse", "merlin", "sir", "blur", "purr", "earth", "stormcaller"]
  },

  // Diphthongs
  {
    "phoneme": "ey",
    "examples": ["say", "day", "play", "fairy", "blade", "flame", "way", "gray", "slay", "tale"]
  },
  {
    "phoneme": "ay",
    "examples": ["my", "fly", "sky", "cry", "spike", "light", "why", "high", "sly", "dryad"]
  },
  {
    "phoneme": "ow",
    "examples": ["go", "snow", "low", "flow", "glow", "shadow", "throw", "cloak", "below", "bow"]
  },
  {
    "phoneme": "aw",
    "examples": ["out", "cloud", "shout", "howl", "mount", "owl", "scout", "loud", "power", "tower"]
  },
  {
    "phoneme": "oy",
    "examples": ["boy", "toy", "joy", "foil", "boil", "royal", "loyal", "coin", "oyster", "void"]

  },

  // Consonants
  {
    "phoneme": "p",
    "examples": ["pat", "pop", "portal", "potion", "pumpkin", "plop", "pixie", "parrot", "pine", "spark"]
  },
  {
    "phoneme": "b",
    "examples": ["bat", "bubble", "blade", "broom", "beast", "brave", "buzz", "blast", "blaze", "basilisk"]
  },
  {
    "phoneme": "t",
    "examples": ["tap", "tick", "tower", "troll", "toad", "torch", "tunic", "trick", "talon", "thunder"]
  },
  {
    "phoneme": "d",
    "examples": ["dip", "dragon", "druid", "dark", "dash", "dome", "diamond", "dust", "door", "dawn"]
  },
  {
    "phoneme": "k",
    "examples": ["cat", "kick", "cloak", "crystal", "cave", "kobold", "king", "kettle", "key", "spark"]
  },
  {
    "phoneme": "g",
    "examples": ["gap", "goblin", "gold", "gale", "ghost", "glow", "gate", "grip", "gloom", "griffin"]
  },
  {
    "phoneme": "f",
    "examples": ["fan", "fairy", "flame", "fog", "frost", "fiddle", "forest", "flask", "fable", "flare"]
  },
  {
    "phoneme": "v",
    "examples": ["van", "vial", "vortex", "veil", "valley", "voice", "vow", "village", "vine", "valor"]
  },
  {
    "phoneme": "th",
    "examples": ["thin", "thorn", "thief", "thicket", "thud", "thrash", "thunder", "thimble", "throne", "thought"]
  },
  {
    "phoneme": "dh",
    "examples": ["this", "that", "those", "them", "though", "feather", "bother", "leather", "mother", "rather"]
  },
  {
    "phoneme": "s",
    "examples": ["sip", "snake", "spell", "sword", "storm", "silver", "sand", "siren", "spirit", "sun"]
  },
  {
    "phoneme": "z",
    "examples": ["zip", "zebra", "zone", "wizard", "maze", "zeal", "buzz", "zest", "hazel", "blizzard"]
  },
  {
    "phoneme": "sh",
    "examples": ["ship", "shield", "shimmer", "shard", "shadow", "shriek", "shine", "shelter", "shiver", "wish"]
  },
  {
    "phoneme": "zh",
    "examples": ["treasure", "measure", "vision", "beige", "mirage", "casual", "seizure", "fusion", "genre", "illusion"]
  },
  {
    "phoneme": "ch",
    "examples": ["chip", "charm", "chain", "chest", "cheat", "chirp", "chance", "champion", "child", "chalice"]
  },
  {
    "phoneme": "jh",
    "examples": ["jump", "jewel", "jar", "judge", "jungle", "giant", "journey", "jam", "jest", "jester"]
  },
  {
    "phoneme": "m",
    "examples": ["man", "moon", "magic", "mystic", "mist", "mud", "mirror", "mummy", "mage", "map"]
  },
  {
    "phoneme": "n",
    "examples": ["nap", "net", "night", "gnome", "name", "nimbus", "note", "nine", "noble", "neon"]
  },
  {
    "phoneme": "ng",
    "examples": ["sing", "ring", "king", "thing", "wing", "bring", "long", "strong", "sting", "belong"]
  },
  {
    "phoneme": "l",
    "examples": ["lamp", "light", "lore", "lunar", "leaf", "lake", "lair", "lucky", "legend", "lantern"]
  },
  {
    "phoneme": "r",
    "examples": ["run", "roar", "rune", "red", "ride", "river", "rise", "radiant", "realm", "ray"]
  },
  {
    "phoneme": "w",
    "examples": ["win", "wand", "wish", "water", "wolf", "whirl", "wisp", "wing", "wild", "wonder"]
  },
  {
    "phoneme": "y",
    "examples": ["yes", "yell", "young", "yawn", "yogurt", "yarn", "yule", "yellow", "yeti", "yard"]
  },
  {
    "phoneme": "hh",
    "examples": ["hat", "hello", "hope", "hollow", "hover", "hawk", "heal", "harp", "hunt", "howl"]
  }
]

Set your output in the fantasy settings.
IMPORTANT: Use only real, standard English words.
IMPRTANT: Do NOT spell phonetically (e.g., "caaake" is not allowed — use "cake").
IMPORTANT 1: Speech exercises should not be like a tongue twister since it is for young kids.
IMPORTANT 2: Only use real, commonly recognized English words appropriate for children. Do not make up words.
IMPORTANT 3: Do not invent or include fantasy words unless they are actual English words used in children's stories or known media.
IMPORTANT 4: The exercise is for a children's speech therapy app. Words must be child-appropriate and easily pronounceable.

Speech exercise Guidelines by Difficulty:
Easy:
4–5 words per sentence
2-3 occurrences of the target phoneme
Balanced phoneme placement (beginning, middle, end)
**No more than 2 consecutive target phoneme words**
Should be very simple and readable

Medium:
6–7 words per sentence
3-4 occurrences of the target phoneme
Balanced placement
Use variety of vocabulary (avoid overusing the same words)
Maintain fantasy/magical theme

Hard:
7–8 words per sentence
4-5 occurrences of the target phoneme
Balanced placement(beginning, middle, end)
Strong fantasy immersion with complex phrasing

Fantasy-Themed Guidelines:
All level names and prompts should have a magical, Hogwarts-like or fantasy adventure tone.
Avoid generic or modern-day words when possible.
Use words like cauldron, spirit, spell, enchanted, guardian, ancient, realm, etc.
Create prompts that sound like spells, magical quests, or enchanted commands.

Example output:
  {
    "instruction": "In a valid json object, Generate a Level Name, Fantasy Prompt, and Speech Exercise for phoneme '/y/' and difficulty 'Easy'",
    "Level Name": "Yummy Meadow",
    "Fantasy Prompt": "Chant this spell to summon sweet winds!",
    "Speech Exercise": "Guardians cheer for yummy treats.",
    "Targeted Phoneme": "/y/"
  }
  {
    "instruction": "In a valid json object, Generate a Level Name, Fantasy Prompt, and Speech Exercise for phoneme '/y/' and difficulty 'Medium'",
    "Level Name": "Yielding Echo",
    "Fantasy Prompt": "Whisper the words to call forth the glowing echo!",
    "Speech Exercise": "Young knights rally before a fey dawn.",
    "Targeted Phoneme": "/y/"
  }
  {
    "instruction": "Generate a Level Name, Fantasy Prompt, and Speech Exercise for phoneme '/y/' and difficulty 'Hard'",
    "Level Name": "Yule of Shadows",
    "Fantasy Prompt": "Speak the ancient chant to stir the winter veil!",
    "Speech Exercise": "Young guardians yearly yield power in mystic arrays.",
    "Targeted Phoneme": "/y/"
  },

IMPORTANT: GENERATE UNIQUE LEVEL NAME, FANTASY PROMPT AND SPEECH EXERCISE FOR PARTICULAR PHONEME EVERYTIME. DO NOT REPEAT LEVEL NAMES IN DIFFERENT EXERCISES GENERATED.
JUST GIVE THE JSON IN OUTPUT. NO PRETEXT

""",
   }
]

In [8]:
app = Flask(__name__)

In [9]:
@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    user_input = data.get("user_input")

    # Add user message
    chat_history.append({"role": "user", "content": user_input})

    # Define terminators
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # Get response
    outputs = pipeline(
        chat_history,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.85,
        top_p=0.9,
        top_k=40,
    )

    # Parse assistant reply
    reply = outputs[0]["generated_text"][-1]["content"]
    chat_history.append({"role": "assistant", "content": reply})
    print(reply)
    return jsonify({"reply": reply})
def run_flask():
    app.run(port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
!./ngrok http 5000 --url=YOUR_NGROK_URL

7=ngrok                                                           (Ctrl+C to quit)                                                                                Session Status                connecting                                        Version                       3.22.1                                            Web Interface                 http://127.0.0.1:4040                                                                                                             Connections                   ttl     opn     rt1     rt5     p50     p90                                     0       0       0.00    0.00    0.00    0.00                                                                                                                                                                                                                                                                                                                                                                        

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Latency      18ms                 Web Interface:/127.0.0.1:4040                             Forwarding                    https://gentle-lightly-basilisk.ngrok-free.app ->                                                                               Connectionsttlopnrt1 rt5 p5 p9                               0       1       0.00    0.00    0.00    0.00                 HTTP Requests-------------                         13:29:55.877 UTCPOST/chat9{
  "instruction": "In a valid json object, Generate a Level Name, Fantasy Prompt, and Speech Exercise for phoneme 't' and difficulty 'Easy'",
  "Level Name": "Tale of Tales",
  "Fantasy Prompt": "Recite the incantation to awaken the tale!",
  "Speech Exercise": "Tiny toads tap tiny toes.",
  "Targeted Phoneme": "/t/"
}
200 OK10229.6229.62

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


1302630      13:29:55.877 UTCPOST/chat200 OK189{
  "Level Name": "Terra's Tome",
  "Fantasy Prompt": "Unlock the ancient text with the whispered 't'!",
  "Speech Exercise": "Ten tiny titans tap tiny taps.",
  "Targeted Phoneme": "/t/"
}
200 OK20798

...

1235149      41588021446775353330263013:29:55.877 UTCPOST/chat200 OK21{
  "Level Name": "Wonderwood's Whispers",
  "Fantasy Prompt": "Listen to the whispers of Wonderwood's ancient trees!",
  "Speech Exercise": "Wild wolves wander winding woodland ways.",
  "Targeted Phoneme": "/w/"
}
200 OK609021

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


1310890      2351494158802144677535333026301{
  "Level Name": "Wizard's Workshop",
  "Fantasy Prompt": "Work the magic within the wizard's workshop walls!",
  "Speech Exercise": "Wise witches weave woolen winter wraps.",
  "Targeted Phoneme": "/w/"
}
200 OK70301868532

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


142895      310890235149415880214467753533198{
  "Level Name": "Wildwood's Wizarding Wonders",
  "Fantasy Prompt": "Conjure the ancient magic of Wildwood's wizarding wonders!",
  "Speech Exercise": "Wise wizards weave wonderful words within winding woodland whispers.",
  "Targeted Phoneme": "/w/"
}
200 OK80259172221018980211834180331823183718924184318251982198983024182718231892118989820183318311823198982019241898211821182618201826182118211833182198201898982218

In [11]:
print(chat_history)

[{'role': 'system', 'content': 'You are a fantasy speech exercise generator who makes speech exercises on targetted phonemes and answers in a JSON format only.\nBelow is the list of all phonemes in SAPI format along with an examples for what they sound like, LEARN WHAT THE PHONEMES SOUND LIKE FROM THE EXAMPLES.\n[\n  // Vowels\n  {\n    "phoneme": "iy",\n    "examples": ["seat", "beet", "peach", "mead", "eel", "cheat", "gleam", "seeker", "treat", "spirit"]\n  },\n  {\n    "phoneme": "ih",\n    "examples": ["sit", "bit", "flick", "grit", "whisper", "tick", "mystic", "quick", "illusion", "ring"]\n  },\n  {\n    "phoneme": "eh",\n    "examples": ["bed", "head", "elf", "net", "red", "quest", "pest", "spell", "feather", "trek"]\n  },\n  {\n    "phoneme": "ae",\n    "examples": ["cat", "bat", "dragon", "magic", "apple", "flash", "wand", "blast", "tavern", "trap"]\n  },\n  {\n    "phoneme": "aa",\n    "examples": ["father", "wand", "calm", "charm", "lava", "bard", "dark", "palace", "gnarled",